In [ ]:
from pathlib import Path

from hydra import compose, initialize_config_dir
from omegaconf import OmegaConf

CONFIG_DIR = (Path.cwd() / "../configs").resolve()
overrides = [
    "trainer.max_epochs=1",
    "trainer.accelerator=cpu",
    "data.split.n_train=2",
    "data.split.n_valid=1",
    "data.split.n_test=1",
]

with initialize_config_dir(version_base=None, config_dir=str(CONFIG_DIR)):
    cfg = compose(config_name="config", overrides=overrides)

print(OmegaConf.to_yaml(cfg))

In [ ]:
from auto_cast.train.autoencoder import build_datamodule


def _get_field(batch, primary: str, fallback: str):
    if hasattr(batch, primary):
        return getattr(batch, primary)
    return getattr(batch, fallback)


datamodule = build_datamodule(cfg.data)
batch = next(iter(datamodule.train_dataloader()))
train_inputs = _get_field(batch, "inputs", "input_fields")
train_outputs = _get_field(batch, "outputs", "output_fields")
train_inputs.shape, train_outputs.shape

In [ ]:
import torch

torch.allclose(train_inputs, train_outputs)

In [ ]:
channel_count = train_inputs.shape[-1]
cfg.model.encoder.in_channels = channel_count
cfg.model.decoder.out_channels = channel_count
print(f"Detected {channel_count} channels; config updated to match input distribution.")

In [ ]:
from hydra.utils import instantiate

from auto_cast.train.autoencoder import build_model

model = build_model(cfg.model)
trainer = instantiate(
    cfg.trainer, logger=False, enable_checkpointing=False, default_root_dir="."
)
model

In [ ]:
trainer.fit(
    model,
    train_dataloaders=datamodule.train_dataloader(),
    val_dataloaders=datamodule.val_dataloader(),
)

In [ ]:
from pathlib import Path

checkpoint_path = Path("notebook_autoencoder.ckpt")
trainer.save_checkpoint(checkpoint_path)
checkpoint_path.resolve()

In [ ]:
import matplotlib.pyplot as plt

device = "cpu"
num_examples = 2

for idx, batch in enumerate(datamodule.test_dataloader()):
    inputs = _get_field(batch, "inputs", "input_fields").to(device)
    outputs, latents = model.forward_with_latent(batch)
    print("Input shape:", inputs.shape)
    print("Output shape:", outputs.shape)
    print("Latent shape:", latents.shape)

    input_frame = inputs[0, 0, :, :, 0].detach().cpu().numpy()
    output_frame = outputs[0, 0, :, :, 0].detach().cpu().numpy()
    latent_frame = latents[0, 0, :, :, 0].detach().cpu().numpy()

    fig, axs = plt.subplots(1, 4, figsize=(10, 4))
    axs[0].imshow(input_frame, cmap="viridis")
    axs[0].set_title("Input")
    axs[1].imshow(output_frame, cmap="viridis")
    axs[1].set_title("Reconstruction")
    axs[2].imshow(output_frame - input_frame, cmap="viridis")
    axs[2].set_title("Difference")
    axs[3].imshow(latent_frame, cmap="viridis")
    axs[3].set_title("Latent")
    plt.tight_layout()
    plt.show()

    if idx + 1 >= num_examples:
        break